In [1]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [2]:
test_meta = pd.DataFrame({'m_view' : [],
                            'm_before' : []
                           })

In [2]:
delay = 3
path = "C:\dev\driver/chromedriver.exe"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}
browser = Chrome(path)
browser.implicitly_wait(delay)

In [3]:
start_url = 'https://www.youtube.com'
browser.get(start_url)
# browser.maximize_window()

In [4]:
browser.find_elements_by_xpath(
    '//*[@id="search-form"]/div/div[1]')[0].click()  # 검색창영역클릭

browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].click()
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys('타다')
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys(Keys.RETURN)  # 엔터

In [5]:
browser.find_elements_by_xpath(
    '//*[@id="container"]/ytd-toggle-button-renderer')[0].click()
browser.find_elements_by_xpath(
    '//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a')[0].click()
# browser.find_elements_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a').send_keys(Keys.RETURN)

In [6]:
body = browser.find_element_by_tag_name('body')  # 스크롤하기 위해 소스 추출

num_of_pagedowns = 10
# 5번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [7]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')

In [10]:
# <ytd-thumbnail class="style-scope ytd-video-renderer" use-hovered-property="">
# 검색에서 주소 가져오기

#############작동코드 건들지 말 것
video_list0 = html.find('div', {'id': 'contents'})
video_list1 = html.find_all('a',{'class': 'yt-simple-endpoint style-scope ytd-video-renderer'})
video_list1
#video_list1 = html.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list2 = video_list0.find_all('div', {'id': 'thumbnail'})
#video_list2 = video_list1.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list0[1].find('a')['href']
#title1 = video_list0[1].find('a',{'id':'video-title'}).text
#title1
#video_list1[0]['href']
#//*[@id="metadata-line"]/span[1]/text()
video_list2 = video_list0.find_all('span', {'class': 'style-scope ytd-video-meta-block'})
video_list2[0]

video_list3 = html.find_all('div', {'class': 'text-wrapper style-scope ytd-video-renderer'})
video_list3[0].find('div',{'id':'metadata-line'})

<div class="style-scope ytd-video-meta-block" id="metadata-line">
<span class="style-scope ytd-video-meta-block">조회수 622회</span>
<span class="style-scope ytd-video-meta-block">16시간 전</span>
<dom-repeat class="style-scope ytd-video-meta-block" strip-whitespace=""><template is="dom-repeat"></template></dom-repeat>
</div>

In [11]:
# 영상 주소 가져오기. 댓글, 좋아요 가져올 때 필요함
base_url = 'https://www.youtube.com'
test_url = []
for i in range(1,30):
    url2 = base_url+video_list1[i]['href']
    test_url.append(url2)

len(test_url)

29

In [12]:
# 조회수 개시 기간
meta_test = video_list3[0].find('div',{'id':'metadata-line'})
view_test = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[0].text.split()[1]
before_hourtest = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[1].text
print(view_test, before_hourtest)

622회 16시간 전


In [13]:
# 저장 테스트
test_meta = pd.DataFrame({'m_view' : [],
                            'm_before' : []
                           })

for i in range(1,20):
    meta_test = video_list3[i].find('div',{'id':'metadata-line'})
    view = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[0].text.split()[1]
    before_hourtest = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[1].text
    test_meta1 = pd.DataFrame({'m_view' : [view],
                            'm_before' : [before_hourtest]
                           })
    test_meta = test_meta.append(test_meta1)

    

In [14]:
test_meta

,m_view,m_before
0,1.3천회,1일 전
0,3.9천회,5일 전
0,408회,1일 전
0,2.3천회,5일 전
0,527회,5일 전
0,1.9천회,6일 전
0,2.3천회,6일 전
0,1.6천회,6일 전
0,1.7천회,6일 전
0,828회,5일 전


In [20]:
## 저장 데이터 + 댓글 개수 좋아요 싫어요 크롤링


## 초기 저장소 설정
test_meta = pd.DataFrame({  'title' : [],
                            'url' : [],
                            'm_view' : [],
                            'm_before' : [],

                          'r_view' : [],
                          'comment' : []
                           })
base_url = 'http://www.youtube.com'


for i in range(0,3):
    # 수집 데이터 반복   
    
    
    title_test = video_list3[i].find('a',{'id':'video-title'}).text
    url_test = base_url+video_list1[i]['href']
    meta_test = video_list3[i].find('div',{'id':'metadata-line'})
    view = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[0].text.split()[1]
    before_hourtest = meta_test.find_all('span',{'class':'style-scope ytd-video-meta-block'})[1].text
    
    # 댓글 개수, 좋아요 싫어요 반영
    
    start_url = test_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(2)
    
    num_of_pagedowns = 4 
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_of_pagedowns -= 1
    
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    
    #start_date = html.find('span',{'class':'style-scope ytd-video-primary-info-renderer'}).text.split()[5]
    r_view = html.find('span',{'class':'view-count style-scope yt-view-count-renderer'})
    comment = html.find('h2',{'id':'count'}).find('yt-formatted-string').text
    
    test_meta1 = pd.DataFrame({'title' : [title_test],
                            'url' : [url_test],
                            'm_view' : [view],
                            'm_before' : [before_hourtest],

                            'r_view' : [r_view],
                            'comment' : [comment]
                           })
    test_meta = test_meta.append(test_meta1)

In [34]:
test_meta[1:]
# 

,title,url,m_view,m_before,r_view,comment
0,\n'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나 / JTBC 뉴스룸\n,http://www.youtube.com/watch?v=t28kEFnJAls,1.3천회,1일 전,"[조회수 3,953회]",댓글 66개
0,\n'타다 금지법' 국회 본회의 통과 / YTN\n,http://www.youtube.com/watch?v=xcgx91Uonxs,3.9천회,5일 전,[조회수 413회],댓글 8개


In [38]:
start_date = html.find('yt-formatted-string',{'class':'style-scope ytd-video-primary-info-renderer'}).text.split()[5]
start_date
#style-scope ytd-video-primary-info-renderer
#style-scope ytd-video-primary-info-renderer

'(2020.03.07/뉴스투데이/MBC)'

In [50]:

comment_data = pd.DataFrame({'youtube_id':[],
                            'comment':[],
                            'like_num':[]})

In [27]:
# 댓글 크롤링 스타트.......ㅠㅠㅠ
# 한개 테스트

start_url = test_url[0] 
browser.get(start_url) 
body = browser.find_element_by_tag_name('body')


In [38]:
num_page_down = 1
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1

In [ ]:
//*[@id="menu"]/a[1]
//*[@id="menu"]

//*[@id="sort-menu"]/yt-sort-filter-sub-menu-renderer/yt-dropdown-menu

//*[@id="menu"]
//*[@id="label"]

//*[@id="sort-menu"]/yt-sort-filter-sub-menu-renderer/yt-dropdown-menu
#menu
#menu
//*[@id="menu"]/a[1]

//*[@id="sort-menu"]

//*[@id="menu"]

//*[@id="menu"]
//*[@id="menu"]/a[1]

//*[@id="menu"]/a[2]/paper-item

//*[@id="dropdown"]

//*[@id="contentWrapper"]
//*[@id="menu"]

//*[@id="menu"]

In [39]:
browser.find_element_by_xpath('//*[@id="sort-menu"]').click()

In [41]:
browser.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click()
# text 선택하게 한다....ㅠㅠ

In [42]:
num_page_down = 20
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1

In [71]:
html_s0 = browser.page_source
html_s = BeautifulSoup(html_s0,'html.parser')

comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
comment0[1].find('yt-formatted-string',{'class':'style-scope ytd-comment-renderer','id':'content-text'}).text

'나이가 들면 변하지 않는다고 하죠?\n꼰대는 대지 맙시다.\n위 교수님 토론 안 나왔으면 좋겠음.'

In [72]:
comment0

[<ytd-comment-renderer class="style-scope ytd-comment-thread-renderer" comment-style="unknown" id="comment">
 <div class="style-scope ytd-comment-renderer" id="body">
 <div class="style-scope ytd-comment-renderer" id="author-thumbnail">
 <a class="yt-simple-endpoint style-scope ytd-comment-renderer" href="/channel/UCcUleKskiM-_k3Isb148EmA">
 <yt-img-shadow class="style-scope ytd-comment-renderer no-transition" fit="" height="40" loaded="" style="background-color: transparent;" width="40"><img alt="박관용" class="style-scope yt-img-shadow" height="40" id="img" src="https://yt3.ggpht.com/a/AATXAJzyJjxmkKbVz7VZ7T0Ji80teUOfK1vxoZWc8g=s48-c-k-c0xffffffff-no-rj-mo" width="40"/></yt-img-shadow>
 </a>
 </div>
 <div class="style-scope ytd-comment-renderer" id="main">
 <div class="style-scope ytd-comment-renderer" id="header">
 <div class="style-scope ytd-comment-renderer" hidden="" id="header-badge">
 <div class="style-scope ytd-comment-renderer" hidden="" id="linked-comment-badge"></div>
 <div cl

In [51]:
for i in range(len(comment0)):
    #댓글
    comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
    
    try:
        aa = comment0[i].find('span',{'id':'vote-count-left'}).text
        #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
        #re.findall('[0-9]',aa)
        #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
        like_num = "".join(re.findall('[0-9]',aa)) + "개"
    except:
        like_num = 0
        
    bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
    youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))
    
    insert_data =  pd.DataFrame({'youtube_id':[youtube_id],
                                               'comment':[comment],
                                               'like_num':[like_num]})

    comment_data = comment_data.append(insert_data)
    
comment_data.index = range(len(comment_data))

In [74]:
comment_data

,youtube_id,comment,like_num
0,박관용,앞으로 자가용으로 영업 해도 되겠네요\n공유경제니까요,17개
1,장수연,나이가 들면 변하지 않는다고 하죠?\n꼰대는 대지 맙시다.\n위 교수님 토론 안 나...,17개
2,김조지문아버,일자리가 필요하면 정상적인 것을찾아 도독질하지말고 대머리선생 타다변로ㆍ 잘해이자식아,14개
3,최동린동린,명색이 교수라면 왜 법을 어기라고 하는가 교수가 학생한데도 법을 지키지 말라고강...,13개
4,배틀그라운드미라마,위정현교수야 그면허 저번달에 영국에서 우버 박탈시켰다 문제가\n더많어 ...,18개
5,최동린동린,외국을 비교하지 마세요 미국은 우리나라 몇배이면 인구가 몇이며 그나라 숫자가 ...,13개
6,김영선,중대교수님 이론은 박사급인데 철학은 F학점입니다.,10개
7,구관옥,렌트카회사가 뭐가필요합니까..위교수 말대로 자가용천진데..노는 자가용으로 렌트카 대...,27개
8,이기만,교수님은 타다반대해서 교수수입보다 조겠는걸요 좀 양심좀 있어봐요 아니 말이되는 ...,16개
9,좋은사람들,공유해서 이익금을 나눠줘야\n공유고뭐고이지\n돈은딴사람이 다가져가는게\n무슨공유경제...,11개


In [ ]:
# 전체

In [77]:
comment_data = pd.DataFrame({'youtube_id':[],
                            'comment':[],
                            'like_num':[]})
browser = Chrome(path)#chrome_options=options
browser.implicitly_wait(delay)

In [78]:
start_url = 'https://www.youtube.com'
browser.get(start_url)
# browser.maximize_window()

In [79]:
browser.find_elements_by_xpath(
    '//*[@id="search-form"]/div/div[1]')[0].click()  # 검색창영역클릭

browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].click()
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys('타다')
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys(Keys.RETURN)  # 엔터

In [80]:
browser.find_elements_by_xpath(
    '//*[@id="container"]/ytd-toggle-button-renderer')[0].click()
browser.find_elements_by_xpath(
    '//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a')[0].click()
# browser.find_elements_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a').send_keys(Keys.RETURN)

In [81]:
body = browser.find_element_by_tag_name('body')  # 스크롤하기 위해 소스 추출

num_of_pagedowns = 10
# 5번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [82]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')

In [83]:
# <ytd-thumbnail class="style-scope ytd-video-renderer" use-hovered-property="">
# 검색에서 주소 가져오기

#############작동코드 건들지 말 것
video_list0 = html.find('div', {'id': 'contents'})
video_list1 = html.find_all('a',{'class': 'yt-simple-endpoint style-scope ytd-video-renderer'})
video_list1
#video_list1 = html.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list2 = video_list0.find_all('div', {'id': 'thumbnail'})
#video_list2 = video_list1.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list0[1].find('a')['href']
#title1 = video_list0[1].find('a',{'id':'video-title'}).text
#title1
#video_list1[0]['href']
#//*[@id="metadata-line"]/span[1]/text()
video_list2 = video_list0.find_all('span', {'class': 'style-scope ytd-video-meta-block'})
video_list2[0]

video_list3 = html.find_all('div', {'class': 'text-wrapper style-scope ytd-video-renderer'})
video_list3[0].find('div',{'id':'metadata-line'})

<div class="style-scope ytd-video-meta-block" id="metadata-line">
<span class="style-scope ytd-video-meta-block">조회수 622회</span>
<span class="style-scope ytd-video-meta-block">17시간 전</span>
<dom-repeat class="style-scope ytd-video-meta-block" strip-whitespace=""><template is="dom-repeat"></template></dom-repeat>
</div>

In [84]:
# 영상 주소 가져오기. 댓글, 좋아요 가져올 때 필요함
base_url = 'https://www.youtube.com'
test_url = []
for i in range(1,30):
    url2 = base_url+video_list1[i]['href']
    test_url.append(url2)

In [85]:
start_url = test_url[0] 
browser.get(start_url) 
body = browser.find_element_by_tag_name('body')

In [86]:
time.sleep(2)

num_page_down = 5
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1

In [88]:
browser.find_element_by_xpath('//*[@id="sort-menu"]').click()

In [89]:
browser.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click()

In [90]:
num_page_down = 20
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1
    
html_s0 = browser.page_source
html_s = BeautifulSoup(html_s0,'html.parser')

In [91]:
comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})

In [92]:
for i in range(len(comment0)):
    #댓글
    comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
    
    try:
        aa = comment0[i].find('span',{'id':'vote-count-left'}).text
        #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
        #re.findall('[0-9]',aa)
        #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
        like_num = "".join(re.findall('[0-9]',aa)) + "개"
    except:
        like_num = 0
        
    bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
    youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))
    
    insert_data =  pd.DataFrame({'youtube_id':[youtube_id],
                                               'comment':[comment],
                                               'like_num':[like_num]})

    comment_data = comment_data.append(insert_data)
    
comment_data.index = range(len(comment_data))

In [94]:
comment_data.index

RangeIndex(start=0, stop=14, step=1)

In [97]:
browser.find_element_by_xpath('//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()

In [148]:

comment_data = pd.DataFrame({'title' :[title],
                            'youtube_id':[youtube_id],
                            'comment':[comment],
                            'like_num':[like_num]
                            })
browser = Chrome(path)#chrome_options=options
browser.implicitly_wait(delay)

In [149]:
for i in range(1,3):
    start_url = test_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(2)
    
    num_page_down = 1
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1
    
    browser.find_element_by_xpath('//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()
    browser.find_element_by_xpath('//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()
    
    num_page_down = 5
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1
        
    html_s0 = browser.page_source
    html_s = BeautifulSoup(html_s0,'html.parser')

    comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})
    title0 = html_s.find_all('ytd-video-primary-info-renderer')
    title = title0[0].find('h1').text.split('/')[0]
    for i in range(len(comment0)):
        
    #댓글
        comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
    
        try:
            aa = comment0[i].find('span',{'id':'vote-count-left'}).text
        #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
        #re.findall('[0-9]',aa)
        #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]',aa)) + "개"
        except:
            like_num = 0
        
        bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))
    
        insert_data =  pd.DataFrame({'title' :[title],
                                     'youtube_id':[youtube_id],
                                     'comment':[comment],
                                     'like_num':[like_num]
                                     })

        comment_data = comment_data.append(insert_data)
    
    comment_data.index = range(len(comment_data))
    
    

In [150]:
comment_data

,title,youtube_id,comment,like_num
0,"타다 베이직, 다음 달 11일부터 잠정 중단",정안,"2심에서 보자, 다시는 껄떡거리지 마라,",0개
1,'타다 금지법' 국회 본회의 통과,모모,난택시가 싢다 왜냐하면 좋은택시 기사도있었지만 난폭운전택시기사 끼어들기 u턴...,3개
2,'타다 금지법' 국회 본회의 통과,김영선,와이티엔은 타다편향 방송 안하고 공정하게 양측 울화통 터지지 않게 겸손한 방송 엄...,1개
3,'타다 금지법' 국회 본회의 통과,윤석,불법 콜택시 타다 사기꾼 이재웅 동등한 경쟁해야지 꽁짜로 쳐묵을려니까 문제지 ...,1개
4,'타다 금지법' 국회 본회의 통과,미스터손,"내 기억속의 택시는 기본요금 나왔다고 카드로 결제한다고 얼굴 찌푸렸던 택시, 물웅덩...",0개
5,'타다 금지법' 국회 본회의 통과,Frienze,중국애들 몰려드네,1개
6,'타다 금지법' 국회 본회의 통과,Kim노옹,택시 표심 겁나서 ㅎㅎㅎ,3개
7,'타다 금지법' 국회 본회의 통과,부엉이오형제,면허증따고 자격증따고 해라\n\n나? 면허없이 면허있는 파일럿보다 비행기 잘모는데 ...,5개
8,'타다 금지법' 국회 본회의 통과,흠흠,저 이해 안 되는 데 요약 좀 해주실 분?,4개
9,'타다 금지법' 국회 본회의 통과,부엉이오형제,국민의 다수가 원한다고 불법을 지양할순없는겁니다. 타다는 애당초 무허가콜택시사업이었...,7개


In [106]:
html_s

<html lang="ko-KR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif; "><head><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRannotations_modulejs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/annotations_module.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/annotations_module.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRendscreenjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/endscreen.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/endscreen.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRcaptionsjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/captions.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/captions.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRminiplayerjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/miniplayer.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/miniplayer.js"></script><script class="

In [128]:
#title0 = html_s.find_all('ytd-video-primary-info-renderer',{'class':'super-title style-scope ytd-video-primary-info-renderer'})
title0 = html_s.find_all('ytd-video-primary-info-renderer')
#video_list2[i].find('a',{'id':'thumbnail'})['href']

In [144]:
title0[0].find('h1').text.split('/')[0]

'타다 베이직, 다음 달 11일부터 잠정 중단 '

In [140]:
html_s

<html lang="ko-KR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif; "><head><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRannotations_modulejs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/annotations_module.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/annotations_module.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRendscreenjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/endscreen.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/endscreen.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRcaptionsjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/captions.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/captions.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflFKcjqko_KRminiplayerjs" data-original-src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/miniplayer.js" src="/yts/jsbin/player_ias-vflFKcjq-/ko_KR/miniplayer.js"></script><script class="

In [139]:
title0

[<ytd-video-primary-info-renderer class="style-scope ytd-watch-flexy" has-date-text_=""><div class="style-scope ytd-video-primary-info-renderer" id="container"><ytd-badge-supported-renderer class="style-scope ytd-video-primary-info-renderer" disable-upgrade="" hidden=""></ytd-badge-supported-renderer><a class="yt-simple-endpoint style-scope ytd-video-primary-info-renderer" hidden=""><yt-icon class="super-title-icon style-scope ytd-video-primary-info-renderer" icon="yt-icons:location_pin"><svg class="style-scope yt-icon" focusable="false" preserveaspectratio="xMidYMid meet" style="pointer-events: none; display: block; width: 100%; height: 100%;" viewbox="0 0 24 24"><g class="style-scope yt-icon">
 <path class="style-scope yt-icon" d="M12 2C8.13 2 5 5.13 5 9c0 5.25 7 13 7 13s7-7.75 7-13c0-3.87-3.13-7-7-7zm0 9.5c-1.38 0-2.5-1.12-2.5-2.5s1.12-2.5 2.5-2.5 2.5 1.12 2.5 2.5-1.12 2.5-2.5 2.5z"></path>
 <path class="style-scope yt-icon" d="M0 0h24v24H0z" fill="none"></path>
 </g></svg>
 </yt-ic